### データベースのサーチの速度比較 

In [1]:
%cd ..

E:\システムトレード入門\predict_git_workspace


In [2]:
import sqlite3
from pathlib import Path
from contextlib import closing
import datetime

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
from line_profiler import LineProfiler

データベースのパス

In [6]:
db_path = Path("db/stock_db/stock.db")

In [7]:
def make_connection():
    conn = sqlite3.connect(db_path,
                           detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES
                           )
    return conn

In [8]:
from get_stock_price import StockDatabase

In [9]:
stock_db = StockDatabase(db_path)

In [10]:
stock_names = ["2531","1679","2607","3470","4284","5199","6308","7088","7952","9048"]

ほしいカラム名からテーブル名を取得する

In [11]:
def get_table_name_column_names_dict(stock_names):
        stock_str_list = ['Open', 'High', 'Low', 'Close', 'Volume']
    
        search_column_names = []
        for stock_name in stock_names:
            stock_name_ohlcv = [i+"_"+ stock_name for i in stock_str_list]  # グローバル変数を使うことに注意
            search_column_names.extend(stock_name_ohlcv)

        search_column_names_array = np.array(search_column_names)        
        
        with closing(make_connection()) as conn:
            # tableの存在確認
            c = conn.cursor()
            
            column_names_str_list = ["'{}'".format(i) for i in search_column_names]  # 引用符を追加
            all_column_names_str = ", ".join(column_names_str_list)
            
            select_sql = "select column_name, arrange_table from column_arrange where column_name in ({})".format(all_column_names_str)  # columnとtable_nameのリスト
            c.execute(select_sql)
            column_table_df = pd.DataFrame(c.fetchall(), columns=["column_name", "table_name"])
            known_column_names_array = column_table_df["column_name"].values  # データベースに存在していたカラム名のarray
                      
            def get_values(df):  # valuesを与えるための関数
                return df.values
            # table名をキーとし，カラム名のarrayをvalueとする辞書,今回はpandasのgroupbyとapplyを利用して求める
            table_name_column_names_dict = dict(column_table_df.groupby("table_name")["column_name"].apply(get_values))  # すでに保存したカラム
            
        return table_name_column_names_dict

In [12]:
stock_names = ["2531","1679","2607","3470","4284","5199","6308","7088","7952","9048"]
get_table_name_column_names_dict(stock_names)

{'table_0': array(['Close_2531', 'High_2531', 'Low_2531', 'Open_2531', 'Volume_2531'],
       dtype=object),
 'table_10': array(['Close_5199', 'High_5199', 'Low_5199', 'Open_5199', 'Volume_5199'],
       dtype=object),
 'table_14': array(['Close_7088', 'High_7088', 'Low_7088', 'Open_7088', 'Volume_7088'],
       dtype=object),
 'table_16': array(['Close_7952', 'High_7952', 'Low_7952', 'Open_7952', 'Volume_7952'],
       dtype=object),
 'table_18': array(['Close_9048', 'High_9048', 'Low_9048', 'Open_9048', 'Volume_9048'],
       dtype=object),
 'table_2': array(['Close_1679', 'High_1679', 'Low_1679', 'Open_1679', 'Volume_1679'],
       dtype=object),
 'table_4': array(['Close_2607', 'High_2607', 'Low_2607', 'Open_2607', 'Volume_2607'],
       dtype=object),
 'table_6': array(['Close_3470', 'High_3470', 'Low_3470', 'Open_3470', 'Volume_3470'],
       dtype=object),
 'table_8': array(['Close_4284', 'High_4284', 'Low_4284', 'Open_4284', 'Volume_4284'],
       dtype=object)}

In [13]:
def get_table_names_column_names(stock_names):
    stock_str_list = ['Open', 'High', 'Low', 'Close', 'Volume']

    search_column_names = []
    for stock_name in stock_names:
        stock_name_ohlcv = [i+"_"+ stock_name for i in stock_str_list]  # グローバル変数を使うことに注意
        search_column_names.extend(stock_name_ohlcv)

    search_column_names_array = np.array(search_column_names)        

    with closing(make_connection()) as conn:
        # tableの存在確認
        c = conn.cursor()

        column_names_str_list = ["'{}'".format(i) for i in search_column_names]  # 引用符を追加
        all_column_names_str = ", ".join(column_names_str_list)

        select_sql = "select column_name, arrange_table from column_arrange where column_name in ({})".format(all_column_names_str)  # columnとtable_nameのリスト
        c.execute(select_sql)
        column_table_df = pd.DataFrame(c.fetchall(), columns=["column_name", "table_name"])
    
    table_name_unique = np.unique(column_table_df["table_name"].values)
    
    return table_name_unique, column_table_df["column_name"].values.copy()

In [14]:
get_table_names_column_names(stock_names)

(array(['table_0', 'table_10', 'table_14', 'table_16', 'table_18',
        'table_2', 'table_4', 'table_6', 'table_8'], dtype=object),
 array(['Close_1679', 'Close_2531', 'Close_2607', 'Close_3470',
        'Close_4284', 'Close_5199', 'Close_7088', 'Close_7952',
        'Close_9048', 'High_1679', 'High_2531', 'High_2607', 'High_3470',
        'High_4284', 'High_5199', 'High_7088', 'High_7952', 'High_9048',
        'Low_1679', 'Low_2531', 'Low_2607', 'Low_3470', 'Low_4284',
        'Low_5199', 'Low_7088', 'Low_7952', 'Low_9048', 'Open_1679',
        'Open_2531', 'Open_2607', 'Open_3470', 'Open_4284', 'Open_5199',
        'Open_7088', 'Open_7952', 'Open_9048', 'Volume_1679',
        'Volume_2531', 'Volume_2607', 'Volume_3470', 'Volume_4284',
        'Volume_5199', 'Volume_7088', 'Volume_7952', 'Volume_9048'],
       dtype=object))

In [15]:
start_datetime = datetime.datetime(2020,11,17,0,0,0)
end_datetime = datetime.datetime(2020,11,18,0,0,0)

###  各テーブルからfor文を利用して取得する

実際に取得はせず，呼び出すだけ

In [21]:
def for_each_table(stock_namers, start_datetime, end_datetime):
    table_name_column_names_dict = get_table_name_column_names_dict(stock_names)
    
    IsFirst = True  # 最初かどうかのフラッグ
    IsMulti = False  # 複数かどうかのフラッグ
    
    for table_name_key in table_name_column_names_dict.keys():
        column_names_array = table_name_column_names_dict[table_name_key]
        
        # 関数の構造上，ここでコネクション
        with closing(make_connection()) as conn:
            sql = """
                  select timestamp, {} from {} where timestamp >= ? and timestamp < ?
                  """.format(", ".join(column_names_array), table_name_key)  # timestampを入れないといけないことに注意

            query_df = pd.read_sql_query(sql,
                                         con=conn,
                                         index_col="timestamp",
                                         params= [start_datetime, end_datetime]
                                         )
        
        if IsFirst:
            left_df = query_df
            IsFirst = False
        else:
            IsMulti = True
            right_df = query_df

        if IsMulti:  # 複数の場合
            left_df = pd.merge(left_df, 
                               right_df,
                               how="outer",
                               left_index=True,
                               right_index=True
                              )
        
    return left_df

In [22]:
query_df = for_each_table(stock_names, start_datetime, end_datetime)
query_df

,Close_2531,High_2531,Low_2531,Open_2531,Volume_2531,Close_5199,High_5199,Low_5199,Open_5199,Volume_5199,...,Close_3470,High_3470,Low_3470,Open_3470,Volume_3470,Close_4284,High_4284,Low_4284,Open_4284,Volume_4284
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-17 00:00:00,1254.0,1254.0,1252.0,1252.0,30400.0,3660.0,3660.0,3640.0,3655.0,11100.0,...,111500.0,111500.0,111500.0,111500.0,6.0,1080.0,1086.0,1080.0,1084.0,1000.0
2020-11-17 00:01:00,1253.0,1254.0,1252.0,1254.0,2500.0,3690.0,3715.0,3660.0,3660.0,2000.0,...,NaN,NaN,NaN,NaN,NaN,1080.0,1080.0,1080.0,1080.0,400.0
2020-11-17 00:02:00,1254.0,1254.0,1251.0,1252.0,2300.0,3685.0,3685.0,3660.0,3660.0,300.0,...,NaN,NaN,NaN,NaN,NaN,1080.0,1087.0,1080.0,1087.0,300.0
2020-11-17 00:03:00,1254.0,1254.0,1252.0,1254.0,2400.0,3660.0,3670.0,3660.0,3670.0,400.0,...,111400.0,111400.0,111400.0,111400.0,2.0,1084.0,1084.0,1084.0,1084.0,200.0
2020-11-17 00:04:00,1249.0,1254.0,1249.0,1254.0,7700.0,3640.0,3640.0,3640.0,3640.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-17 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


実行時間で6秒かかっている

In [23]:
def profile_for_each_table():
    for_each_table(stock_names, start_datetime, end_datetime)

prf = LineProfiler()                                                                                                                                                                                  
prf.add_function(for_each_table)                                                                                      
prf.runcall(profile_for_each_table)                                                                                            
prf.print_stats()

Timer unit: 1e-07 s

Total time: 7.82727 s
File: <ipython-input-21-76736bf466cf>
Function: for_each_table at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def for_each_table(stock_namers, start_datetime, end_datetime):
     2         1    7598059.0 7598059.0      9.7      table_name_column_names_dict = get_table_name_column_names_dict(stock_names)
     3                                               
     4         1         78.0     78.0      0.0      IsFirst = True  # 最初かどうかのフラッグ
     5         1         51.0     51.0      0.0      IsMulti = False  # 複数かどうかのフラッグ
     6                                               
     7        10       1883.0    188.3      0.0      for table_name_key in table_name_column_names_dict.keys():
     8         9        444.0     49.3      0.0          column_names_array = table_name_column_names_dict[table_name_key]
     9                                                   
    10  

### joinを用いて取得

まず，ベースとなるsqlを実行してみる．

In [23]:
sql="""
select
table_0.timestamp, Open_2531, Open_2607, Open_4284
from
table_0
left join
table_4
on
table_0.timestamp = table_4.timestamp
left join
table_8
on
table_0.timestamp = table_8.timestamp
where
table_0.timestamp >= ? and table_0.timestamp < ?
"""
with closing(make_connection()) as conn:
    query_df = pd.read_sql_query(sql,
                                 con=conn,
                                 index_col="timestamp",
                                 params= [start_datetime, end_datetime]
                                )
query_df

,Open_2531,Open_2607,Open_4284
timestamp,,,
2020-11-17 00:00:00,1252.0,3035.0,1084.0
2020-11-17 00:01:00,1254.0,3010.0,1080.0
2020-11-17 00:02:00,1252.0,3010.0,1087.0
2020-11-17 00:03:00,1254.0,3015.0,1084.0
2020-11-17 00:04:00,1254.0,3030.0,NaN
...,...,...,...
2020-11-17 23:55:00,NaN,NaN,NaN
2020-11-17 23:56:00,NaN,NaN,NaN
2020-11-17 23:57:00,NaN,NaN,NaN


In [54]:
def all_join_tables(stock_names, start_datetime, end_datetime):
    table_name_column_names_dict = get_table_name_column_names_dict(stock_names)
    
    all_table_names_rest = list(table_name_column_names_dict.keys())
    all_column_names = np.concatenate(list(table_name_column_names_dict.values()),axis=0)
    
    first_table_name = all_table_names_rest.pop(0)  # 最初だけ取得
    def add_timestamp(table_name):
        return table_name+".timestamp"
    
    column_names_str = add_timestamp(first_table_name)+", "+ ", ".join(all_column_names)
    join_str = ""
    for table_name in all_table_names_rest:
        join_str += "\nleft join {} \non \n{} = {}".format(table_name, add_timestamp(first_table_name), add_timestamp(table_name))

    where_str = "\nwhere \n{} >= ? and {} < ?".format(add_timestamp(first_table_name), add_timestamp(first_table_name))

    sql = "select {} \nfrom {} {} {}".format(column_names_str, first_table_name, join_str, where_str)
    
    with closing(make_connection()) as conn:
        query_df = pd.read_sql_query(sql,
                                     con=conn,
                                     index_col="timestamp",
                                     params= [start_datetime, end_datetime]
                                    )
    return query_df

In [55]:
query_df = all_join_tables(stock_names, start_datetime, end_datetime)
query_df

,Close_2531,High_2531,Low_2531,Open_2531,Volume_2531,Close_5199,High_5199,Low_5199,Open_5199,Volume_5199,...,Close_3470,High_3470,Low_3470,Open_3470,Volume_3470,Close_4284,High_4284,Low_4284,Open_4284,Volume_4284
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-17 00:00:00,1254.0,1254.0,1252.0,1252.0,30400.0,3660.0,3660.0,3640.0,3655.0,11100.0,...,111500.0,111500.0,111500.0,111500.0,6.0,1080.0,1086.0,1080.0,1084.0,1000.0
2020-11-17 00:01:00,1253.0,1254.0,1252.0,1254.0,2500.0,3690.0,3715.0,3660.0,3660.0,2000.0,...,NaN,NaN,NaN,NaN,NaN,1080.0,1080.0,1080.0,1080.0,400.0
2020-11-17 00:02:00,1254.0,1254.0,1251.0,1252.0,2300.0,3685.0,3685.0,3660.0,3660.0,300.0,...,NaN,NaN,NaN,NaN,NaN,1080.0,1087.0,1080.0,1087.0,300.0
2020-11-17 00:03:00,1254.0,1254.0,1252.0,1254.0,2400.0,3660.0,3670.0,3660.0,3670.0,400.0,...,111400.0,111400.0,111400.0,111400.0,2.0,1084.0,1084.0,1084.0,1084.0,200.0
2020-11-17 00:04:00,1249.0,1254.0,1249.0,1254.0,7700.0,3640.0,3640.0,3640.0,3640.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-17 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
def profile_all_join_tables():
    for i in range(10):
        all_join_tables(stock_names, start_datetime, end_datetime)

prf = LineProfiler()                                                                                                                                                                                  
prf.add_function(all_join_tables)                                                                                      
prf.runcall(profile_all_join_tables)                                                                                            
prf.print_stats()

Timer unit: 1e-07 s

Total time: 17.1858 s
File: <ipython-input-54-a223c26f9dfe>
Function: all_join_tables at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def all_join_tables(stock_names, start_datetime, end_datetime):
     2        10   67184780.0 6718478.0     39.1      table_name_column_names_dict = get_table_name_column_names_dict(stock_names)
     3                                               
     4        10       1577.0    157.7      0.0      all_table_names_rest = list(table_name_column_names_dict.keys())
     5        10       3396.0    339.6      0.0      all_column_names = np.concatenate(list(table_name_column_names_dict.values()),axis=0)
     6                                               
     7        10       1528.0    152.8      0.0      first_table_name = all_table_names_rest.pop(0)  # 最初だけ取得
     8        10        515.0     51.5      0.0      def add_timestamp(table_name):
     9          

### ビューを作成したのち，取得 

#### ビューの作成 

In [16]:
sql="""
create view view_2531_2607_4284 as
select
table_0.timestamp, Open_2531, Open_2607, Open_4284
from
table_0
left join
table_4
on
table_0.timestamp = table_4.timestamp
left join
table_8
on
table_0.timestamp = table_8.timestamp
"""

with closing(make_connection()) as conn:
    c = conn.cursor()
    c.execute(sql)

In [25]:
sql="""
create view view_2531_2607_4284 as
select
table_0.timestamp, Open_2531, Open_2607, Open_4284
from
table_0
left join
table_4
on
table_0.timestamp = table_4.timestamp
left join
table_8
on
table_0.timestamp = table_8.timestamp
where 
table_0.timestamp >= '2020-11-17 00:00:00'
and
table_0.timestamp <  '2020-11-18 00:00:00'
"""

with closing(make_connection()) as conn:
    c = conn.cursor()
    c.execute(sql)

#### ビューから取得 

In [27]:
"{}".format(start_datetime)

'2020-11-17 00:00:00'

In [26]:
sql = """
select timestamp, Open_2531, Open_2607, Open_4284 from view_2531_2607_4284 where timestamp >= ? and timestamp < ?
"""    
with closing(make_connection()) as conn:
    query_df = pd.read_sql_query(sql,
                                 con=conn,
                                 index_col="timestamp",
                                 params= [start_datetime, end_datetime]
                                )
query_df

,Open_2531,Open_2607,Open_4284
timestamp,,,
2020-11-17 00:00:00,1252.0,3035.0,1084.0
2020-11-17 00:01:00,1254.0,3010.0,1080.0
2020-11-17 00:02:00,1252.0,3010.0,1087.0
2020-11-17 00:03:00,1254.0,3015.0,1084.0
2020-11-17 00:04:00,1254.0,3030.0,NaN
...,...,...,...
2020-11-17 23:55:00,NaN,NaN,NaN
2020-11-17 23:56:00,NaN,NaN,NaN
2020-11-17 23:57:00,NaN,NaN,NaN


かなり高速だが，まだ1秒弱かかる

In [30]:
with closing(make_connection()) as conn:
    c = conn.cursor()
    c.execute("select type, name from sqlite_master")
    table_list = c.fetchall()
    print(table_list)

[('table', 'meta_data'), ('table', 'column_arrange'), ('index', 'sqlite_autoindex_column_arrange_1'), ('table', 'column_count'), ('index', 'sqlite_autoindex_column_count_1'), ('table', 'table_0'), ('index', 'sqlite_autoindex_table_0_1'), ('table', 'table_1'), ('index', 'sqlite_autoindex_table_1_1'), ('table', 'table_2'), ('index', 'sqlite_autoindex_table_2_1'), ('table', 'table_3'), ('index', 'sqlite_autoindex_table_3_1'), ('table', 'table_4'), ('index', 'sqlite_autoindex_table_4_1'), ('table', 'table_5'), ('index', 'sqlite_autoindex_table_5_1'), ('table', 'table_6'), ('index', 'sqlite_autoindex_table_6_1'), ('table', 'table_7'), ('index', 'sqlite_autoindex_table_7_1'), ('table', 'table_8'), ('index', 'sqlite_autoindex_table_8_1'), ('table', 'table_9'), ('index', 'sqlite_autoindex_table_9_1'), ('table', 'table_10'), ('index', 'sqlite_autoindex_table_10_1'), ('table', 'table_11'), ('index', 'sqlite_autoindex_table_11_1'), ('table', 'table_12'), ('index', 'sqlite_autoindex_table_12_1'), 

In [31]:
with closing(make_connection()) as conn:
    c = conn.cursor()
    c.execute("select type, name from sqlite_master where type='view' and name = '{}'".format("view_table_0_table_2_table_4_table_6_table_8_table_10_table_14_table_16_table_18"))
    table_list = c.fetchall()
    print(table_list)

[('view', 'view_table_0_table_2_table_4_table_6_table_8_table_10_table_14_table_16_table_18')]


In [32]:
with closing(make_connection()) as conn:
    c = conn.cursor()
    c.execute("select type, name from sqlite_master where type='view' and name = '{}'".format("table_0_table_2_table_4_table_6_table_8_table_10_table_14_table_16_table_18"))
    table_list = c.fetchall()
    print(table_list is None)

False


In [24]:
with closing(make_connection()) as conn:
    c = conn.cursor()
    c.execute("drop view view_2531_2607_4284")

In [35]:
def make_view_name(table_names_array):
    table_names_array = np.unique(table_names_array)
    def split_table(table_name):
        table, number = table_name.split("_")
        return table_name, number
    np_split_table = np.frompyfunc(split_table, 1, 2)
    table_names_array, table_name_numbers = np_split_table(table_names_array)
    table_name_numbers = np.argsort(table_name_numbers.astype("int"))
    
    table_names_array = table_names_array[table_name_numbers].copy()
    return "view_{}".format("_".join(table_names_array))

def create_view(stock_names):
    table_name_column_names_dict = get_table_name_column_names_dict(stock_names)
    all_table_names_rest = list(table_name_column_names_dict.keys())
    # 全てのテーブル名がある段階でviewの名前を作成する
    view_name = make_view_name(all_table_names_rest)

    all_column_names = np.concatenate(list(table_name_column_names_dict.values()),axis=0)
    
    first_table_name = all_table_names_rest.pop(0)  # 最初だけ取得
    def add_timestamp(table_name):
        return table_name+".timestamp"
    
    column_names_str = add_timestamp(first_table_name)+", "+ ", ".join(all_column_names)
    join_str = ""
    for table_name in all_table_names_rest:
        join_str += "\nleft join {} \non \n{} = {}".format(table_name, add_timestamp(first_table_name), add_timestamp(table_name))
        
    sql = "create view {} as\nselect {} \nfrom {} {}".format(view_name, column_names_str, first_table_name, join_str)
    print(sql)
    with closing(make_connection()) as conn:
        c = conn.cursor()
        c.execute(sql)

create_view(stock_names)

create view view_table_0_table_2_table_4_table_6_table_8_table_10_table_14_table_16_table_18 as
select table_0.timestamp, Close_2531, High_2531, Low_2531, Open_2531, Volume_2531, Close_5199, High_5199, Low_5199, Open_5199, Volume_5199, Close_7088, High_7088, Low_7088, Open_7088, Volume_7088, Close_7952, High_7952, Low_7952, Open_7952, Volume_7952, Close_9048, High_9048, Low_9048, Open_9048, Volume_9048, Close_1679, High_1679, Low_1679, Open_1679, Volume_1679, Close_2607, High_2607, Low_2607, Open_2607, Volume_2607, Close_3470, High_3470, Low_3470, Open_3470, Volume_3470, Close_4284, High_4284, Low_4284, Open_4284, Volume_4284 
from table_0 
left join table_10 
on 
table_0.timestamp = table_10.timestamp
left join table_14 
on 
table_0.timestamp = table_14.timestamp
left join table_16 
on 
table_0.timestamp = table_16.timestamp
left join table_18 
on 
table_0.timestamp = table_18.timestamp
left join table_2 
on 
table_0.timestamp = table_2.timestamp
left join table_4 
on 
table_0.timesta

In [46]:
def get_from_view(stock_names, start_dateitme, end_datetime):
    table_name_column_names_dict = get_table_name_column_names_dict(stock_names)
    
    all_table_names = list(table_name_column_names_dict.keys())
    all_column_names = np.concatenate(list(table_name_column_names_dict.values()),axis=0)
    
    view_name = make_view_name(all_table_names)
    
    column_names_str = ", ".join(all_column_names)
    
    sql = "select timestamp, {} from {} where timestamp >= ? and timestamp < ?".format(column_names_str, view_name)

    with closing(make_connection()) as conn:
        query_df = pd.read_sql_query(sql,
                                     con=conn,
                                     index_col="timestamp",
                                     params= [start_datetime, end_datetime]
                                    )
    return query_df

In [47]:
get_from_view(stock_names, start_datetime, end_datetime)

,Close_2531,High_2531,Low_2531,Open_2531,Volume_2531,Close_5199,High_5199,Low_5199,Open_5199,Volume_5199,...,Close_3470,High_3470,Low_3470,Open_3470,Volume_3470,Close_4284,High_4284,Low_4284,Open_4284,Volume_4284
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-17 00:00:00,1254.0,1254.0,1252.0,1252.0,30400.0,3660.0,3660.0,3640.0,3655.0,11100.0,...,111500.0,111500.0,111500.0,111500.0,6.0,1080.0,1086.0,1080.0,1084.0,1000.0
2020-11-17 00:01:00,1253.0,1254.0,1252.0,1254.0,2500.0,3690.0,3715.0,3660.0,3660.0,2000.0,...,NaN,NaN,NaN,NaN,NaN,1080.0,1080.0,1080.0,1080.0,400.0
2020-11-17 00:02:00,1254.0,1254.0,1251.0,1252.0,2300.0,3685.0,3685.0,3660.0,3660.0,300.0,...,NaN,NaN,NaN,NaN,NaN,1080.0,1087.0,1080.0,1087.0,300.0
2020-11-17 00:03:00,1254.0,1254.0,1252.0,1254.0,2400.0,3660.0,3670.0,3660.0,3670.0,400.0,...,111400.0,111400.0,111400.0,111400.0,2.0,1084.0,1084.0,1084.0,1084.0,200.0
2020-11-17 00:04:00,1249.0,1254.0,1249.0,1254.0,7700.0,3640.0,3640.0,3640.0,3640.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-17 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-17 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
def profile_all_join_tables():
    for i in range(10):
        get_from_view(stock_names, start_datetime, end_datetime)

prf = LineProfiler()                                                                                                                                                                                  
prf.add_function(get_from_view)                                                                                      
prf.runcall(profile_all_join_tables)                                                                                            
prf.print_stats()

Timer unit: 1e-07 s

Total time: 16.7559 s
File: <ipython-input-46-a1fddc9c48a4>
Function: get_from_view at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_from_view(stock_names, start_dateitme, end_datetime):
     2        10   63292699.0 6329269.9     37.8      table_name_column_names_dict = get_table_name_column_names_dict(stock_names)
     3                                               
     4        10       1461.0    146.1      0.0      all_table_names = list(table_name_column_names_dict.keys())
     5        10       3212.0    321.2      0.0      all_column_names = np.concatenate(list(table_name_column_names_dict.values()),axis=0)
     6                                               
     7        10      40567.0   4056.7      0.0      view_name = make_view_name(all_table_names)
     8                                               
     9        10       2199.0    219.9      0.0      column_names_st

In [57]:
with closing(make_connection()) as conn:
    c = conn.cursor()
    c.execute("drop view view_table_0_table_2_table_4_table_6_table_8_table_10_table_14_table_16_table_18")

In [61]:
with closing(make_connection()) as conn:
    c = conn.cursor()
    c.execute("drop view view_table_0_table_2_table_4_table_6_table_8_table_10_table_14_table_16_table_18")